### Small lung cancer data analysis

In [1]:
import pandas as pd

# Load the CSV file into a pandas DataFrame
data2_python = pd.read_csv("/Users/sisiphozinja/Documents/GitHub/cell-translation/data/tracerX.csv")

# Display the first few rows to verify
print(data2_python.head())

    cruk_id tumour_id_muttable_cruk tumour_id_per_patient  age     sex  \
0  CRUK0034                CRUK0034              CRUK0034   68  Female   
1  CRUK0150                CRUK0150              CRUK0150   81    Male   
2  CRUK0159                CRUK0159              CRUK0159   60    Male   
3  CRUK0090                CRUK0090              CRUK0090   65    Male   
4  CRUK0045                CRUK0045              CRUK0045   85    Male   

        ethnicity  cigs_perday  years_smoking  packyears  \
0    White- Irish         20.0             35     35.000   
1  White- British         44.5             49    109.025   
2  White- British         20.0             38     38.000   
3  White- British         10.0             35     17.500   
4  White- British         10.0             25     12.500   

  smoking_status_merged  ...  os_time  cens_dfs dfs_time cens_dfs_any_event  \
0             Ex-Smoker  ...     1849         0     1849                  0   
1             Ex-Smoker  ...     136

I actually need Transcriptome Profiling dataset, such as 3b00356a-e947-42f1-8b5f-6dcd789d9272.rna_seq.star_splice_junctions.tsv.gz from NIH from Project	
APOLLO-LUAD.

Maybe?
https://www.ncbi.nlm.nih.gov/gene/3845